# Segmenting and Clustering Neighborhoods in Toronto Part-1

## Question: 1

### Import important libraries

In [1]:
from bs4 import BeautifulSoup
import requests   # library to handle requests
import lxml       # parse the website in lxml format
import numpy as np
import pandas as pd

In [72]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


### Scraping website  & wrangle and dump  scrapped Data


In [13]:

List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [15]:
soup = BeautifulSoup(source, 'xml')

table=soup.find('table')


In [16]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [18]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [19]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


###  Data Cleaning (remove rows where Borough is 'Not assigned')

In [21]:
df=df[df['Borough']!='Not assigned']


In [23]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [25]:
df_merge = pd.merge(df, temp_df, on='Postalcode')


In [26]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)


In [27]:
df_merge.drop_duplicates(inplace=True)


In [28]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)


In [42]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [30]:
df_merge.shape


(103, 3)

## Question: 2

In [31]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [32]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [37]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood', 'Latitude','Longitude']]


In [40]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


## Question: 3

In [73]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [74]:

geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [75]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [76]:

CLIENT_ID = 'JTUZX0DERA2ZQVUDKDQHEWJ25JBTCKF0EE0105NK0GDKPBH4' # your Foursquare ID
CLIENT_SECRET = 'LCRS1P0LRUSNKCTRKQMMUH52Y3WSFCUVYALYFPDGGIUCIQYI' # your Foursquare Secret
VERSION = '20180605'

In [77]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

The Beaches, The Beaches
The Danforth West, Riverdale, The Danforth West, Riverdale
India Bazaar, The Beaches West, India Bazaar, The Beaches West
Studio District, Studio District
Lawrence Park, Lawrence Park
Davisville North, Davisville North
North Toronto West,  Lawrence Park, North Toronto West,  Lawrence Park
Davisville, Davisville
Moore Park, Summerhill East, Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park, Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale, Rosedale
St. James Town, Cabbagetown, St. James Town, Cabbagetown
Church and Wellesley, Church and Wellesley
Regent Park, Harbourfront, Regent Park, Harbourfront
Garden District, Ryerson, Garden District, Ryerson
St. James Town, St. James Town
Berczy Park, Berczy Park
Central Bay Street, Central Bay Street
Richmond, Adelaide, King, Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands, Harbourfront East, Union Station, Toronto Islands
To

In [79]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches, The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches, The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,"The Beaches, The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,"The Beaches, The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Beaches, The Beaches",43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [80]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Berczy Park, Berczy Park",58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place, Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport, CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
"Central Bay Street, Central Bay Street",64,64,64,64,64,64
"Christie, Christie",16,16,16,16,16,16
"Church and Wellesley, Church and Wellesley",75,75,75,75,75,75
"Commerce Court, Victoria Hotel, Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Davisville North, Davisville North",10,10,10,10,10,10


In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1637, 231)

In [82]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Berczy Park, Berczy Park",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,"Central Bay Street, Central Bay Street",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.015625


In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Berczy Park, Berczy Park",Coffee Shop,Seafood Restaurant,Cheese Shop,Bakery,Cocktail Bar,Farmers Market,Café,Restaurant,Beer Bar,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place, ...",Café,Breakfast Spot,Coffee Shop,Intersection,Bar,Bakery,Restaurant,Climbing Gym,Burrito Place,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Fast Food Restaurant,Farmers Market,Burrito Place,Auto Workshop,Spa,Restaurant,Brewery,Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Plane
4,"Central Bay Street, Central Bay Street",Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Bar,Japanese Restaurant,Department Store


## Making Clusters for Neighborhood

In [86]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [87]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031,0,Asian Restaurant,Trail,Pub,Health Food Store,Donut Shop,Doner Restaurant,Dog Run,Dumpling Restaurant,Cupcake Shop,Distribution Center
41,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Bakery,Pub,Pizza Place
42,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572,0,Fish & Chips Shop,Pub,Sandwich Place,Board Shop,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop
43,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790,4,Photography Studio,Park,Swim School,Bus Line,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [88]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Berczy Park, Berczy Park",Coffee Shop,Seafood Restaurant,Cheese Shop,Bakery,Cocktail Bar,Farmers Market,Café,Restaurant,Beer Bar,Pharmacy
1,0,"Brockton, Parkdale Village, Exhibition Place, ...",Café,Breakfast Spot,Coffee Shop,Intersection,Bar,Bakery,Restaurant,Climbing Gym,Burrito Place,Italian Restaurant
2,0,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Fast Food Restaurant,Farmers Market,Burrito Place,Auto Workshop,Spa,Restaurant,Brewery,Park
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Plane
4,0,"Central Bay Street, Central Bay Street",Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Bar,Japanese Restaurant,Department Store


In [90]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


### Creating Map

In [91]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters